## Создание искусственных нейронных сетей

### Импорт собранных и обработанных данных

In [7]:
import pandas as pd

# Для mac
df_nlp = pd.read_csv(r'/Users/user/Documents/ML_Classic.csv')

# Для win
#df_nlp = pd.read_csv(r'C:\Users\User\Downloads\ML.csv'), если в начале лишних столбец с индексами

df_ml = df_nlp.drop(df_nlp.columns[[0,3,4,5,6,7,8,9,10]], axis = 1)

df_ml

,main_rubric,rubric,text_lemm,title_lemm
0,Происшествия,Происшествия,следствие попадать глава аналитический отдел п...,обвинение взяточничество арестовывать подмоско...
1,Происшествия,Происшествия,систематическое получение взятка арестовывать ...,систематическое получение взятка арестовывать ...
2,Происшествия,Происшествия,место выезжать аварийный бригада сотрудник адм...,приграничное село белгородский область попадат...
3,Происшествия,Происшествия,село старый хутор белгородский область попадат...,украинские войско обстреливать село белгородск...
4,Происшествия,Происшествия,четверг декабрь обстрел вооруженный формирован...,губернатор гладков всу обстреливать село стары...
...,...,...,...,...
3391,Наука,Космос,ученые томский университет система управление ...,томские ученый разрабатывать система электропи...
3392,Наука,Космос,вес бортовой система космический аппарат число...,томске разрабатывать система электропитание сп...
3393,Наука,Космос,двадцать пятый август год зарегистрировать мар...,ученые зафиксировать марсотрясение рекордный силы
3394,Наука,Космос,сейсмометр марсианский модуль insight зафиксир...,марсе происходить крупный землетрясение истори...


### Импорт необходимых библиотек

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.preprocessing import OneHotEncoder

### TensorFlow

In [18]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

# from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score

# from gensim.models import Word2Vec

from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, Input, GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras import backend as K
# from keras.engine.topology import Layer

In [38]:
# Темы
df_topic = df_ml['main_rubric']


# Разделение на выборки
rubrics = ['Политика', 'Общество', 'Экономика', 'В мире', 'Спорт', 'Происшествия', 'Культура', 'Технологии', 'Наука']

encoder = LabelEncoder()
rubrics_list = df_ml['main_rubric'].to_list()
rubric_labels = encoder.fit_transform(rubrics_list)

X = df_ml['text_lemm']
y = rubric_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
my_tags = rubrics

df_train = X_train
df_test = X_test

df = df_ml
print("Количество уникальных тем: ", len(df['main_rubric'].unique()))
print(len(df['text_lemm']))
#print(df['text_lemm'].tolist())


# Избавляемся от пустых строк
df = df[df['text_lemm'].notna()]
print(df_train)
print(df_test)


max_words = 0
for desc in df_train:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном описании заявки: {} слов'.format(max_words))


# Максимальное количество слов
num_words = 197

# Количество классов (тем)
nb_classes = len(df_topic.tolist())

posts_train = df_train
posts_test = df_test

# Максимально количество слов в тексте
max_post_len = 35


# Преобразуем классы в векторный вид
y_train
#= to_categorical(df_train - 1, nb_classes)
#y_test = to_categorical(df_test - 1, nb_classes)

Количество уникальных тем:  9
3396
2864    нескольких месяц слух airpods max начинать пос...
994     проекты реализовать нацпроект экология рамка р...
464     великобритании призывать киев сильно бояться з...
1740    правоохранительные орган испания задерживать л...
1074    исполнительный директор альянс туристический а...
                              ...                        
1095    это повлиять неудобство новый ограничение стал...
1130    ямале автоинспектор перекрывать дорога ради бе...
1294    станислав  —  расти число российский называть ...
860     женщина постоянно чувствовать боль жировик дав...
3174    медики колумбийский университет устанавливать ...
Name: text_lemm, Length: 2377, dtype: object
291     официальный представитель кремль дмитрий песок...
3056    например сквозной шифрование доступно больший ...
2093    экс нападающий ряд футбольный клуб россия браз...
432     всу продолжать терять командный состав донбасс...
479     фоне близкий поставка американец зрк patri

array([7, 3, 4, ..., 8, 3, 2])

In [36]:
# Темы
df_topic = df_ml['main_rubric']


# Разделение на выборки
rubrics = ['Политика', 'Общество', 'Экономика', 'В мире', 'Спорт', 'Происшествия', 'Культура', 'Технологии', 'Наука']

encoder = LabelEncoder()
rubrics_list = df_ml['main_rubric'].to_list()
rubric_labels = encoder.fit_transform(rubrics_list)

X = df_ml['text_lemm']
y = rubric_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
my_tags = rubrics

df_train = X_train
df_test = X_test

df = df_ml
print("Количество уникальных тем: ", len(df['main_rubric'].unique()))
print(len(df['text_lemm']))
#print(df['text_lemm'].tolist())


# Избавляемся от пустых строк
df = df[df['text_lemm'].notna()]
print(df_train)
print(df_test)


max_words = 0
for desc in df_train:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном описании заявки: {} слов'.format(max_words))


# Максимальное количество слов
num_words = 10000

# Количество классов (тем)
nb_classes = len(df_topic.tolist())

posts_train = df_train
posts_test = df_test

# Максимально количество слов в тексте
max_post_len = 35

# Преобразуем классы в векторный вид
y_train = to_categorical(df_train, nb_classes)
y_test = to_categorical(df_test, nb_classes)

# Производим токенизацию текста
tokenizer = Tokenizer(num_words=num_words)  # 10000 самых встречаемых слов
#tokenizer = Tokenizer()

tokenizer.fit_on_texts(df['text_lemm'].tolist())
print(len(tokenizer.index_word))
#num_words = len(tokenizer.index_word) - 1

# Слова в виде чисел
sequences_train = tokenizer.texts_to_sequences(posts_train.tolist())
sequences_test = tokenizer.texts_to_sequences(posts_test.tolist())

# Преобразуем векторы к одной длине путем добавления нулей
x_train = pad_sequences(sequences_train, maxlen=max_post_len)
x_test = pad_sequences(sequences_test, maxlen=max_post_len)

'''
word_index = tokenizer.word_index

tokens = []
for text in df_train['text_lemm']:
    token = text.split(' ')
    token = [tok for tok in token if tok != "" and tok != '\n']
    tokens.append(token)

df_train['tokens'] = tokens
data = df_train['tokens']
print(data)
w2v_model = Word2Vec(data, min_count=1, size=100, window=5)

embedding_index = dict()
for word in w2v_model.wv.vocab:
    embedding_index[word] = w2v_model.wv.word_vec(word)
print(embedding_index)


def prepare_matrix(embedding_dict, emb_size=100):
    num_words = len(word_index)
    embedding_matrix = np.zeros((num_words, emb_size))

    for word, i in tqdm(word_index.items()):
        if i >= num_words:
            continue

        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec
    return embedding_matrix


embedding_matrix = prepare_matrix(embedding_index)
num_words = len(word_index)
'''


def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results


x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)
# print(x_train)

#print(x_train)
#print(tokenizer.word_index)
#print(len(x_train))

#smote = SMOTE(sampling_strategy='auto',
#              random_state=0,
#              k_neighbors=4)
#X_train, y_train = smote.fit_resample(x_train, y_train)
#X_test, y_test = smote.fit_resample(x_test, y_test)
#print(len(X_sm))
#print(len(X_train))

# Сеть LSTM ############################################################

model = Sequential()

'''
model.add(Embedding(num_words, 256, input_length=max_post_len))
model.add(Dropout(0.5))
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(Dense(207, activation='softmax'))
'''
'''
model.add(Embedding(num_words, 256, input_length=max_post_len))
model.add(Dropout(0.5))
model.add(LSTM(256, recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(210, activation='softmax'))
'''

model.add(Dense(256, activation='relu', input_shape=(10000,)))
model.add(Dropout(0.2))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(207, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(nb_classes, activation='softmax'))
model.summary()


'''
model.add(Embedding(num_words, 202, input_length=max_post_len))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(207, activation='softmax'))
'''
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.load_weights('model1.h5')
model_save_path = 'model.h5'
checkpoint_callback = ModelCheckpoint(model_save_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

history = model.fit(x_train,
                    y_train,
                    epochs=7,
                    batch_size=128,
                    validation_split=0.1,
                    callbacks=[checkpoint_callback])

score = model.evaluate(x_test, y_test, batch_size=128, verbose=1)

print()
print("Оценка теста: {}".format(score[0]))
print("Оценка точности модели: {}".format(score[1]))


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
#plt.legend()
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()


Количество уникальных тем:  9
3396
2864    нескольких месяц слух airpods max начинать пос...
994     проекты реализовать нацпроект экология рамка р...
464     великобритании призывать киев сильно бояться з...
1740    правоохранительные орган испания задерживать л...
1074    исполнительный директор альянс туристический а...
                              ...                        
1095    это повлиять неудобство новый ограничение стал...
1130    ямале автоинспектор перекрывать дорога ради бе...
1294    станислав  —  расти число российский называть ...
860     женщина постоянно чувствовать боль жировик дав...
3174    медики колумбийский университет устанавливать ...
Name: text_lemm, Length: 2377, dtype: object
291     официальный представитель кремль дмитрий песок...
3056    например сквозной шифрование доступно больший ...
2093    экс нападающий ряд футбольный клуб россия браз...
432     всу продолжать терять командный состав донбасс...
479     фоне близкий поставка американец зрк patri

ValueError: invalid literal for int() with base 10: 'нескольких месяц слух airpods max начинать поступать покупатель год назад устройство похоже считаться устаревать airpods pro второй поколение добавлять ряд технологический инновация который airpods m

### Содание нейронной сети без использования библиотек

In [ ]:
import random
import numpy as np

INPUT_DIM = 1
OUT_DIM = 9
H_DIM = 5

#dataset = [(df_ml['text_lemm'][i][None, ...], df_ml['rubric'][i]) for i in range(len(df_ml['rubric']))]

W1 = np.random.rand(INPUT_DIM, H_DIM)
b1 = np.random.rand(1, H_DIM)
W2 = np.random.rand(H_DIM, OUT_DIM)
b2 = np.random.rand(1, OUT_DIM)

W1 = (W1 - 0.5) * 2 * np.sqrt(1/INPUT_DIM)
b1 = (b1 - 0.5) * 2 * np.sqrt(1/INPUT_DIM)
W2 = (W2 - 0.5) * 2 * np.sqrt(1/H_DIM)
b2 = (b2 - 0.5) * 2 * np.sqrt(1/H_DIM)

#Функция активации
def relu(t):
    return np.maximum(t,0)

def softmax(t):
    out = np.exp(t)
    return out / np.sum(out)

def softmax_batch(t):
    out = np.exp(t)
    return out / np.sum(out, axis=1, keepdims=True)

def sparse_cross_entropy(z,y):
    return -np.log(z[0,y])

def sparse_cross_entropy_batch(z,y):
    return -np.log(np.array([z[j, y[j]] for j in range(len(y))]))

def to_full(y, num_classes):
    y_full = np.zeros((1, num_classes))
    y_full[0, y] = 1
    return y_full

def to_full_batch(y, num_classes):
    y_full = np.zeros((len(y), num_classes))
    for j, yj in enumerate(y):
        y_full[j, yj] = 1
    return y_full

def relu_deriv(t):
    return (t >= 0).astype(float)

#Гиперпараметры нейросети
#Скорость обучения (Learning rate)
ALPHA = 0.001
#Количество эпох
NUM_EPOCHS = 100
#Размер батча
BATCH_SIZE = 1


loss_arr = []

for ep in range(NUM_EPOCHS):
    random.shuffle(dataset)
    for i in range(len(X_train) // BATCH_SIZE):
        #batch_x, batch_y = zip(*dataset[i*BATCH_SIZE : i*BATCH_SIZE+BATCH_SIZE])
        #x = np.concatenate(batch_x, axis=0)
        #y = np.array(batch_y)
        
        #Forward propagation (прямое распространение)
        t1 = x @ W1 + b1
        h1 = relu(t1)
        t2 = h1 @ W2 + b2
        z = softmax_batch(t2)
        E = sparse_cross_entropy_batch(z,y)

        #Back propagation (обратное распространение)
        y_full = to_full_batch(y, OUT_DIM)
        dE_dt2 = z - y_full
        dE_dW2 = h1.T @ dE_dt2
        dE_db2 = np.sum(dE_dt2, axis=0, keepdims=True)
        dE_dh1 = dE_dt2 @ W2.T
        dE_dt1 = dE_dh1 * relu_deriv(t1)
        dE_dW1 = x.T @ dE_dt1
        dE_db1 = np.sum(dE_dt1, axis=0, keepdims=True)

        #Update (обновление весов)
        W1 = W1 - ALPHA * dE_dW1
        b1 = b1 - ALPHA * dE_db1
        W2 = W2 - ALPHA * dE_dW2
        b2 = b2 - ALPHA * dE_db2
        W1 = W1 - ALPHA * dE_dW1
        probs = predict(x)
        pred_class = np.argmax(probs)
        class_names = ['Политика', 'Общество', 'Экономика', 'В мире', 'Спорт', 'Происшествия', 'Культура', 'Технологии', 'Наука']

        #Метрика "зависимость ошибки от итерации"
        loss_arr.append(E)
        
        #Метрика "точность предсказания" ...
        
def predict(x):
    t1 = x @ W1 + b1
    h1 = relu(t1)
    t2 = h1 @ W2 + b2
    z = softmax_batch(t2)
    return z

def calc_accuracy():
    correct = 0
    for x,y in dataset:
        z = predict(x)
        y_pred = np.argmax(z)
        if y_pred == y:
            correct += 1
    acc = correct / len(dataset)
    return acc

accuracu = calc_accuracy()
print('Accuracy:', accuracy)

#График падения ошибки
import matplotlib.pyplot as plt
plt.plot(loss_arr)
plt.show()